# Trabajo integrador - Parte 1
## Python y Numpy

**Nombre**: Nahuel Severini

In [1]:
import numpy as np

## Ejercicio 1

Dada una matriz en formato *numpy array*, donde cada fila de la matriz representa un vector matemático, se requiere computar las normas $l_0$, $l_1$, $l_2$, $l_{\infty}$, según la siguientes definiciones:

\begin{equation}
    ||\mathbf{x}||^{p} = \bigg(\sum_{j=1}^{n}{|x_i|^p}\bigg)^{\frac{1}{p}}
\end{equation}

con los casos especiales para $p=0$ y $p=\infty$ siendo:

\begin{equation}
    \begin{array}{rcl}
        ||\mathbf{x}||_0 & = & \bigg(\sum_{j=1 \wedge x_j != 0}{|x_i|}\bigg)\\
        ||\mathbf{x}||_{\infty} & = & \max_{i}{|x_i|}\\
    \end{array}
\end{equation}

In [2]:
def calculate_norm(matrix: np.ndarray, ord: int) -> np.ndarray:
    if ord == 0:
        return np.sum(np.absolute(matrix), axis=1)
    elif ord == np.inf:
        return np.max(np.absolute(matrix), axis=1)
    else:
        return np.sum(np.absolute(matrix) ** ord, axis=1) ** (1/ord)
    
matrix = np.array([[0, -1, 2], [3, 4, 5], [6, 7, 8]])

print("Matrix:\n", matrix, "\n")

print("Norm L0: ", calculate_norm(matrix, 0))
print("Norm L1: ", calculate_norm(matrix, 1))
print("Norm L2: ", calculate_norm(matrix, 2))
print("Norm L infinity: ", calculate_norm(matrix, np.inf))


Matrix:
 [[ 0 -1  2]
 [ 3  4  5]
 [ 6  7  8]] 

Norm L0:  [ 3 12 21]
Norm L1:  [ 3. 12. 21.]
Norm L2:  [ 2.23606798  7.07106781 12.20655562]
Norm L infinity:  [2 5 8]


## Ejercicio 2

En clasificación contamos con dos arreglos, la “verdad” y la “predicción”. Cada elemento de los arreglos pueden tomar dos valores, “True” (representado por 1) y “False” (representado por 0). Entonces podemos definir 4 variables:

* True Positive (TP): El valor verdadero es 1 y el valor predicho es 1
* True Negative (TN): El valor verdadero es 0 y el valor predicho es 0
* False Positive (FP): El valor verdadero es 0 y el valor predicho es 1
* False Negative (FN): El valor verdadero es 1 y el valor predicho es 0

A partir de esto definimos:

* Precision = TP / (TP + FP)
* Recall = TP / (TP + FN)
* Accuracy = (TP + TN) / (TP + TN + FP + FN)
 
Calcular las 3 métricas con Numpy y operaciones vectorizadas.

In [3]:
truth = np.array([1,1,0,1,1,1,0,0,0,1])
prediction = np.array([1,1,1,1,0,0,1,1,0,0])

def get_true_positive(a: np.ndarray, b: np.ndarray) -> int:
    arr = np.logical_and(a,b)
    return np.count_nonzero(arr)

def get_true_negative(a: np.ndarray, b: np.ndarray) -> int:
    arr = np.logical_or(a,b)
    return np.count_nonzero(np.invert(arr))

def get_false_positive(a: np.ndarray, b: np.ndarray) -> int:
    arr = np.logical_xor(a,b)
    arr = np.logical_and(arr,b)
    return np.count_nonzero(arr)

def get_false_negative(a: np.ndarray, b: np.ndarray) -> int:
    arr = np.logical_xor(a,b)
    arr = np.logical_and(arr,a)
    return np.count_nonzero(arr)

TP = get_true_positive(truth, prediction)
TN = get_true_negative(truth, prediction)
FP = get_false_positive(truth, prediction)
FN = get_false_negative(truth, prediction)

print("Precision: ", TP/(TP+FP))
print("Recall: ", TP/(TP+FN))
print("Accuracy: ", (TP+TN)/(TP+TN+FP+FN))

Precision:  0.5
Recall:  0.5
Accuracy:  0.4


## Ejercicio 3

Crear una función que separe los datos en train-validation-test. Debe recibir de parametros:

- X: Array o Dataframe que contiene los datos de entrada del sistema.
- y: Array o Dataframe que contiene la(s) variable(s) target del problema.
- train_percentage: _float_ el porcentaje de training.
- test_percentage: _float_ el porcentaje de testing.
- val_percentage: _float_ el porcentaje de validación.
- shuffle: _bool_ determina si el split debe hacerse de manera random o no.

Hints: 

* Usar Indexing y slicing
* Usar np.random.[...]

In [80]:
def split(X_input,
          Y_input,
          train_size=0.7,
          val_size=0.15,
          test_size=0.15,
          random_state=42,
          shuffle=True):
    
        if shuffle:
            np.random.seed(random_state)
            shuffled_indices = np.random.permutation(len(X_input))
        else:
            shuffled_indices = np.arange(len(X_input))
        
        total_samples = len(X_input)
        train_end = int(total_samples * train_size)
        val_end = int(total_samples * (train_size + val_size))
        
        train_indices = shuffled_indices[:train_end]
        val_indices = shuffled_indices[train_end:val_end]
        test_indices = shuffled_indices[val_end:]
        
        X_train = X_input[train_indices]
        Y_train = Y_input[train_indices]
        
        X_val = X_input[val_indices]
        Y_val = Y_input[val_indices]
        
        X_test = X_input[test_indices]
        Y_test = Y_input[test_indices]
        
        return X_train, X_val, X_test, Y_train, Y_val, Y_test


array_x = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
array_y = np.array([0, 1, 1, 1, 4, 5, 1, 7, 1, 1])

print("Split function executed without shuffle:\n", split(array_x, array_y, shuffle=False))
print("Split function executed with shuffle:\n", split(array_x, array_y))

Split function executed without shuffle:
 (array([0, 1, 2, 3, 4, 5, 6]), array([7]), array([8, 9]), array([0, 1, 1, 1, 4, 5, 1]), array([7]), array([1, 1]))
Split function executed with shuffle:
 (array([8, 1, 5, 0, 7, 2, 9]), array([4]), array([3, 6]), array([1, 1, 5, 0, 7, 1, 1]), array([4]), array([1, 1]))
